In [ ]:
# Decorators
%reload_ext autoreload
%autoreload 2
%matplotlib notebook

use_cuda = False
#General
import torch
import os, glob

if use_cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
import torchvision
import numpy as np

#Used for data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

#Used to define densenet
import torch.nn as nn
from torchvision import models
from torchvision.models.densenet import DenseNet
from torchvision.models.densenet import OrderedDict
from torchvision.models.densenet import _DenseBlock
from torchvision.models.densenet import _DenseLayer
from torchvision.models.densenet import _Transition
import torch.nn.functional as F

#Used for optimization
import torch.optim as optim

#timing
import contexttimer

# Data directory
data_dir = '/Users/zfphil/datasets/motiondeblur/learning_data/'

import matplotlib.pyplot as plt

## Define neural network

In [ ]:
class FresnelNet(nn.Module):
    def __init__(self, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0, num_classes=1000, num_channels = 3, crop_size = (0,0)):
        super(FresnelNet, self).__init__()
        # Define structures
        self.down_features = []
        self.up_features = []        
        
        num_features = num_init_features
        #First convolution (convolves to features)
        self.first_layer = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(num_channels, num_features, kernel_size=7, stride=1, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('conv1', nn.Conv2d(num_features, num_features, kernel_size=3, stride=1, padding=1, bias=False)),
            ('norm1', nn.BatchNorm2d(num_features)),
            ('relu1', nn.ReLU(inplace=True)),            
        ]))
        # Convolve again to reduce size
        self.second_layer = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(num_features, num_features, kernel_size=3, stride=2, padding=1, bias=False)),
            ('norm0', nn.BatchNorm2d(num_features)),
            ('relu0', nn.ReLU(inplace=True)),
        ]))
        
        # Compare with results from first convolution to produce final output
        self.last_layer = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(num_init_features*2, 1, kernel_size=3, stride=1, padding=1, bias=False)),
            ('relu0', nn.ReLU(inplace=True)),
            ('norm0', nn.BatchNorm2d(1)),
        ]))        
        
        # Cropping layer
        self.crop_size = (-1 * crop_size[0], -1 * crop_size[1])
        
        # Down-up pair
        for i, num_layers in enumerate(block_config):
            # Define denseblock
            down_block = nn.Sequential(OrderedDict([
            ('pool0', nn.AvgPool2d(kernel_size=2, stride=2)),
            ('denseblock%d' % (i + 1), _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)),
            ]))
            num_features_old = num_features
            num_features += num_layers * growth_rate
            
            # Transition layer is composed of bath-norm, relu, and a conv layer
            # Transition layer is not needed for the last denseblock
            if i != len(block_config) - 1:
                down_block.add_module('norm', nn.BatchNorm2d(num_features))
                down_block.add_module('relu', nn.ReLU(inplace=True))
                down_block.add_module('conv0', nn.Conv2d(num_features, num_features // 2,
                                          kernel_size=1, stride=1, bias=False))
                num_features = num_features // 2    
            else:
                down_block.add_module('final_bn', nn.BatchNorm2d(num_features))
            self.down_features.append(down_block)
            
            # Upsampling block is composed of upsampling, relu, batchnorm, and conv layer.
            up_block = nn.Sequential(OrderedDict([('deconv0',nn.Conv2d(num_features_old + num_features, num_features_old + num_features,
                                          kernel_size=5, stride=1, bias=False, padding = 2))]))
            up_block.add_module('relu_adj0', nn.ReLU(inplace=True))
            up_block.add_module('norm_adj0', nn.BatchNorm2d(num_features_old + num_features))

            up_block.add_module('deconv1', nn.Conv2d(num_features_old + num_features, num_features_old,
                                              kernel_size=1, stride=1, bias=False))
            up_block.add_module('relu_adj1', nn.ReLU(inplace=True))
            up_block.add_module('norm_adj1', nn.BatchNorm2d(num_features_old))
            self.up_features.append(up_block)
        
    def forward(self, x):
        features_save = []
        features = self.first_layer(x)
#         print(features.shape)
        features_save.append(features)            
        features = self.second_layer(features)
#         print(features.shape)
        features_save.append(features)            

        for block in self.down_features:
            features = block(features)
#             print(features.shape)
            features_save.append(features)
        features = features_save.pop()
        
        for block_idx in range(len(self.up_features)):
            block = self.up_features[-1 - block_idx]
            features = F.interpolate(features, scale_factor = 2)
            pop_features = features_save.pop()
            features = torch.cat([features, pop_features],dim = 1)
            features = block(features)
        features = F.interpolate(features, scale_factor = 2)
        pop_features = features_save.pop()
        features = torch.cat([features, pop_features],dim = 1)
        features = self.last_layer(features)          
        features = F.pad(features, self.crop_size, "constant", 0)
        return features                        

### Create neural network instance
For now, default parameters

In [ ]:
num_channels = 1
crop_size = (0,40)
fresnel_net = FresnelNet(num_channels = num_channels, crop_size = crop_size)
if use_cuda:
    fresnel_net = fresnel_net.cuda()

## Load Data

In [ ]:
class FresnelDataset(Dataset):
    def __init__(self, data, transform=None):
        """
        Args:
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.transform = transform
        self.data = data
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Define output directory
output_directory = "%s/%s"   % (os.getcwd(), "data/learning_data/")
print(output_directory)
# Find frames
files = list(glob.glob(data_dir + '*.npz'))
assert len(files) > 0
files.sort()
data_full = []
for frame_index in range(len(files)):    
    # Load data point (second line deals with weird structuring of .npz files)
    _data = dict(np.load(files[frame_index]))
    data = {key:_data[key].item() for key in _data}['arr_0']

    input_data = np.real(data['measurements'][0]['array']).astype('float32')
    output_data = np.real(data['ground_truth']['array']).astype('float32')
    data_pair = (input_data[np.newaxis,0:128,0:128], output_data[np.newaxis,0:128,0:88])
    data_full.append(data_pair)
    if frame_index % 100 == 0:
        print('Loaded file %d' % (frame_index))

In [ ]:
fresnel_data = FresnelDataset(data_full)
dataloader   = DataLoader(fresnel_data, batch_size=10, shuffle=True)

## Define cost function & optimizer

In [ ]:
criterion = nn.MSELoss()
# optimizer = optim.SGD(fresnel_net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(fresnel_net.parameters())

## Train network

In [ ]:
num_epochs = 1
with contexttimer.Timer() as timer:
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(dataloader, 0):
            # get the inputs
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            if use_cuda:
                inputs = inputs.cuda()
                labels = labels.cuda()
            outputs = fresnel_net(inputs)
#             print(outputs.shape)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            del outputs
            if use_cuda:
                torch.cuda.empty_cache()
        print('Finished epoch %d, time used: %f, error: %f' % (epoch, timer.elapsed, running_loss))
    print('Finished Training, time used:', timer.elapsed)

In [ ]:
outputs = fresnel_net(inputs)

In [ ]:
outdata = outputs.cpu().detach().numpy()
inputdata = inputs.cpu().detach().numpy()
labeldata = labels.cpu().detach().numpy()

In [ ]:
outdata.shape

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(131)
plt.imshow(np.squeeze(np.real(inputdata)[3,:,:,:]))
plt.title('input')
plt.subplot(132)
plt.imshow(np.squeeze(np.real(labeldata)[3,:,:,:]))
plt.title('ground truth')
plt.subplot(133)
plt.imshow(np.squeeze(np.real(outdata)[3,:,:,:]))
plt.title('output')

In [ ]:
torch.save(fresnel_net, output_directory+'network.pth.tar')

In [ ]:
import scipy.io as sio

In [ ]:
results = {"input": np.real(inputdata), "gt": np.real(labeldata), "output": np.real(outdata)}
sio.savemat("results.mat", results)